**Authors:** Jeff Hansen, Dylan Skinner, Jason Vasquez, Dallin Stewart

## Imports

In [1]:
# python imports
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from copy import deepcopy
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sbn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib.collections import LineCollection

# Native imports
from py_files.features import generate_features
from py_files.data_manager import get_X_y, get_nyc_gdf

## Lasso Feature Selection

In [ ]:
def lasso_feature_selection(X, y):
    """
    Performs feature selection using the LassoLarsIC method

    Parameters
    - X (dataframe): dataframe of input features
    - y (series): series of target values

    Returns:
    - (dict): dictionary of results (optimal alpha, optimal BIC, lasso coefficients, important features)
    """
    lasso_lars_ic = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC(criterion="bic", normalize=False)).fit(X, y)

    results = pd.DataFrame(
        {
            "alphas": lasso_lars_ic[-1].alphas_,
            "BIC criterion": lasso_lars_ic[-1].criterion_,
        }
    ).set_index("alphas")

    optimal_alpha = results[results['BIC criterion'] == results['BIC criterion'].min()].index

    # Train a Lasso model with the optimal alpha for feature selection
    lasso = linear_model.Lasso(alpha=optimal_alpha)
    lasso.fit(X, y)

    return {'Optimal Alpha': optimal_alpha.values[0], 'Optimal BIC': results.loc[optimal_alpha].values[0].tolist()[0],
            'Lasso Coeffs': lasso.coef_.round(4), 'Important Features': X.columns[lasso.coef_ != 0].values}

### Lasso Regression Model

In [ ]:
def lasso_regression_model(optimal_alpha):
    """
    This function takes in the optimal alpha from the Lasso Lars IC Feature Selection and trains a Lasso Regression
    model on the important features.

    Parametes:
    - optimal_alpha (float): The optimal alpha from the Lasso Lars IC Feature Selection

    Returns:
    - Dictionary with the RMSE of the model
    """
    # Important features selected from Lasso Lars IC Feature Selection
    important_features = ['pickup_minute', 'distance_km', 'temperature_2m (°C)', 'cloudcover (%)', 'avg_cluster_duration']

    # Create dataframe of important features
    X2 = X[important_features]

    # Get test train split
    X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)

    # Create and fit the model.
    model = linear_model.Lasso(alpha=optimal_alpha)
    model.fit(X_train, y_train)

    # Predict on test data and compute RMSE
    y_pred = model.predict(X_test)
    return {'RMSE': mean_squared_error(y_test, y_pred, squared=False)}

## LightGBM Hyperparameter Selection

In [ ]:
def lightgbm_hyperparameter_search(X, y):
    """
    Performs hyperparameter search for LightGBM model

    Parameters:
    - X (dataframe): dataframe of input features
    - y (dataframe): dataframe of target variables

    Returns:
    - Dictionary of best parameters and best RMSE
    """
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Create param grid
    param_grid = {
        'boosting_type': ['gbdt', 'dart'],
        'num_leaves': [30, 40],
        'learning_rate': [0.01, 0.05],
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'reg_alpha': [0.1, 0.5],
        'reg_lambda': [0.1, 0.5],
    }

    # LightGBM
    lgb_train = lgb.LGBMRegressor()

    # Grid search
    grid_search = GridSearchCV(estimator=lgb_train, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=1)
    grid_search.fit(X_train, y_train)

    # Validate
    y_pred = grid_search.predict(X_test)

    return {'Best parameters from grid search': grid_search.best_params_, 'Best RMSE': mean_squared_error(y_test, y_pred, squared=False)}

## XGBoost Hyperparameter Selection

In [4]:
def xgboost_hyperparameter_search(X, y):
    """
    Performs a grid search on the XGBoost model
    
    Parameters:
    - X (DataFrame): The input features
    - y (Series): The target variable

    Returns:
    - None
    """
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    param_grid = {
        'booster': ['gbtree', 'dart'],
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.05],
        'max_depth': [10, 20],
        'alpha': [0.1, 0.5],
    }

    # XGBoost
    xgb_model = xgb.XGBRegressor()

    # Grid search
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=1)
    grid_search.fit(X_train, y_train)

    # Best params
    print('Best parameters from grid search: ', grid_search.best_params_)

### XGBoost Model

In [8]:
def xgboost_model(X, y):
    """
    XGBoost model with optimal hyperparameters
    
    Parameters:
    - None

    Returns:
    - Dictionary of RMSE results
    """
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # XGBoost
    xgb_model = xgb.XGBRegressor(booster='gbtree', n_estimators=100, learning_rate=0.01, max_depth=10, alpha=0.1)

    # Fit
    xgb_model.fit(X_train, y_train)

    # Validate
    y_pred = xgb_model.predict(X_test)

    return {'RMSE': mean_squared_error(y_test, y_pred, squared=False)}

## Random Forest Hyperparameter Selection

In [ ]:
def random_forest_gridsearch():
    """
    Performs a hyperparameter gridsearch with cross validation
    to find the optimal parameters for a RandomForestRegressor
    
    Parameters:
    - None

    Returns:
    - None
    """
        
    # get the X and y, and add the features
    X, y = get_X_y(force_clean=True)
    feature_X = generate_features(X, y)

    # drop the pickup datetime feature since sklearn RandomForest does
    # not accept datetime columns
    feature_X = feature_X.drop(columns=['pickup_datetime'])

    # get the X and y for training
    X_train = feature_X.copy()
    y_train = y.copy()
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)

    # to speed up the grid search, we will use the first four instances
    # of each cluster-to-cluster pair of data points
    df = X_train.copy()
    df = df.sort_values(by='avg_cluster_duration')

    dfs = []

    sample_per_class = 4
    for _ in range(sample_per_class):
        firsts = df['avg_cluster_duration'] != df['avg_cluster_duration'].shift(1)
        dfs.append(df.loc[firsts].copy())
        df = df.loc[~firsts].copy()

    # combine all of the reprsentative samples
    final_df = pd.concat(dfs, axis=0).sort_values('avg_cluster_duration')

    # shuffle the data so that it is no longer sorted by avg_cluster_duration
    X_train = final_df.copy().sample(frac=1)
    y_train = y_train.loc[X_train.index]

    X_train = X_train.values.astype(np.float32)
    y_train = y_train.values.astype(np.float32)

    # perform the RandomForest gridsearch to find the best
    # hyperparameters
    param_grid = {
        'n_estimators': [100, 200, 400, 800, 1000],
        'max_depth': [None, 3, 5, 10, 20],
        'max_features': ['sqrt', 'log2'],
        'min_samples_leaf': [1, 2, 3, 5],
    }
    rf = RandomForestRegressor(warm_start=False)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, verbose=3, n_jobs=-2, cv=4).fit(X_train, y_train)
    best_params = grid_search.best_params_

    # save the grid_search_model for future loading
    with open("models/rf_grid_search.pkl", "wb") as f:
        pickle.dump(grid_search, f)
        
    # print the best parameters
    print("Best parameters RandomForest:", best_params)
    
    # train a model and compute the RMSE on the test set
    X_train, X_test, y_train, y_test = train_test_split(feature_X, y, test_size=0.2, random_state=42)
    
    rf = RandomForestRegressor(**best_params)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    rf_rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    print("RandomForest RMSE:", rf_rmse)
    

## KMeans Clustering

In [ ]:
def kmeans_pickup_dropoff_model(df, n_clusters=200):
    """
    Fits KMeans models for pickup and dropoff locations, labels each location by its cluster,
    computes the average duration between each cluster, and merges it onto the original dataframe.

    Parameters:
    - df (DataFrame): The input DataFrame containing pickup and dropoff locations.
    - n_clusters (int): The number of clusters for KMeans.

    Returns:
    - DataFrame: The modified DataFrame with cluster labels and average cluster duration.
    """
    # fit the kmeans models and label each pickup and dropoff location by its cluster
    kmeans_pickup = (KMeans(n_clusters=n_clusters)
        .fit(df.loc[:, ['pickup_longitude', 'pickup_latitude']].values))
    kmeans_dropoff = (KMeans(n_clusters=n_clusters)
        .fit(df.loc[:, ['dropoff_longitude', 'dropoff_latitude']].values))
    df['pickup_cluster'] = kmeans_pickup.predict(df[['pickup_longitude', 'pickup_latitude']].values)
    df['dropoff_cluster'] = kmeans_dropoff.predict(df[['dropoff_longitude', 'dropoff_latitude']].values)

    # compute the average duration between each cluster and merge this onto the original dataframe
    group_durations = (df
        .groupby(['pickup_cluster', 'dropoff_cluster'])['trip_duration']
        .mean()
        .reset_index()
        .rename(columns={'trip_duration': 'avg_cluster_duration'}))
    df = pd.merge(
        left=df, right=group_durations, how='left',
        left_on=['pickup_cluster', 'dropoff_cluster'], right_on=['pickup_cluster', 'dropoff_cluster'])

    # fill the missing values with the mean of the average duration from cluster to cluster
    df['avg_cluster_duration'] = df['avg_cluster_duration'].fillna(df['avg_cluster_duration'].mean())
    df.drop(columns=['pickup_200_cluster', 'dropoff_200_cluster', 'trip_duration'], inplace=True)
    return df

### KMeans Clustering Plot

In [ ]:
# constants/parameters for this code cell
SHOW_PLOTS = True
LOAD_SAVED_KMEANS_MODELS = True

# load in the cleaned training data and the NYC geopandas dataframe
# with all of the NYC streets
X, y = get_X_y(force_clean=True)
nyc_gdf = get_nyc_gdf()


#########################
# PLOT PICKUP LOCATIONS #
#########################
def plot_pickup_locations(X):
    """
    Plots the NYC streets and pickup locations as a scatter plot on top of the streets.

    Parameters:
    - X (DataFrame): The DataFrame containing pickup locations.

    Returns:
    - None
    """
    # plot the nyc streets
    plt.gcf()
    nyc_gdf.plot(linewidth=0.1, edgecolor='black', figsize=(12, 12), alpha=0.5, label="NYC Streets")

    # plot the pickup locations as a scatter plot on top of the nyc streets
    plt.scatter(X['pickup_longitude'], X['pickup_latitude'], c='red', alpha=0.75, s=0.1, label="Pickup Locations")
    leg = plt.legend(loc='upper left')
    for lh in leg.legend_handles: 
        lh.set_alpha(1)
    plt.title("Pickup Locations")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # save the plot
    plt.savefig("images/pickup_locations_save.png")
    plt.show() if SHOW_PLOTS else plt.clf()


##########################
# PLOT DROPOFF LOCATIONS #
##########################
def plot_dropoff_locations(X):
    """
    Plots the NYC streets and dropoff locations as a scatter plot on top of the streets.

    Parameters:
    - X (DataFrame): The DataFrame containing dropoff locations.

    Returns:
    - None
    """
    # plot the nyc streets
    plt.gcf()
    nyc_gdf.plot(linewidth=0.1, edgecolor='black', figsize=(12, 12), alpha=0.5, label="NYC Streets")

    # plot the dropoff locations as a scatter plot on top of the nyc streets
    plt.scatter(X['dropoff_longitude'], X['dropoff_latitude'], c='green', alpha=0.75, s=0.1, label="Dropoff Locations")
    leg = plt.legend(loc='upper left')
    for lh in leg.legend_handles: 
        lh.set_alpha(1)
    plt.title("Dropoff Locations")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # save the plot
    plt.savefig("images/dropoff_locations_save.png")
    plt.show() if SHOW_PLOTS else plt.clf()


#####################
# KMEANS CLUSTERING #
#####################
def kmeans_pickup_dropoff_predict(df, n_clusters=200):
    """
    Applies KMeans clustering to predict pickup and dropoff locations, or loads pre-trained models if available.

    Parameters:
    - df (DataFrame): The DataFrame containing pickup and dropoff locations.
    - n_clusters (int): The number of clusters for KMeans. Default is 200.

    Returns:
    - df (DataFrame): The DataFrame with predicted clusters for pickup and dropoff locations.
    - pickup_200_centers (array): The cluster centers for pickup locations.
    - dropoff_200_centers (array): The cluster centers for dropoff locations.
    """
    df = deepcopy(X)

    # load kmeans_pickup and kmeans_dropoff from the models folder using pickle
    if LOAD_SAVED_KMEANS_MODELS:
        with open("models/kmeans_200_pickup.pkl", "rb") as file:
            kmeans_200_pickup = pickle.load(file)
        with open("models/kmeans_200_dropoff.pkl", "rb") as file:
            kmeans_200_dropoff = pickle.load(file)
        
            
    # fit kmeans_pickup and kmeans_dropoff with 200 clusters
    else:
        n_clusters = 200
        kmeans_pickup = (KMeans(n_clusters=n_clusters)
            .fit(df.loc[:, ['pickup_longitude', 'pickup_latitude']].values))
        kmeans_dropoff = (KMeans(n_clusters=n_clusters)
            .fit(df.loc[:, ['dropoff_longitude', 'dropoff_latitude']].values))
        
        # save the models to pickle files for loading later
        with open("models/kmeans_200_pickup.pkl", "wb") as file:
            pickle.dump(kmeans_pickup, file)
        with open("models/kmeans_200_dropoff.pkl", "wb") as file:
            pickle.dump(kmeans_dropoff, file)

    # predict the clusters for each pickup and dropoff location
    df['pickup_200_cluster'] = kmeans_200_pickup.predict(df[['pickup_longitude', 'pickup_latitude']].values)
    df['dropoff_200_cluster'] = kmeans_200_dropoff.predict(df[['dropoff_longitude', 'dropoff_latitude']].values)

    # get the centers
    pickup_200_centers = kmeans_200_pickup.cluster_centers_
    dropoff_200_centers = kmeans_200_dropoff.cluster_centers_
    return df, pickup_200_centers, dropoff_200_centers


#######################################
# PLOT PICKUP LOCATIONS WITH CLUSTERS #
#######################################
def plot_cluster_pickup(df, pickup_200_centers):
    """
    Plots KMeans clustering for pickup locations.

    Parameters:
    - df (DataFrame): The DataFrame containing pickup locations and their associated clusters.
    - pickup_200_centers (array): The cluster centers for pickup locations.

    Returns:
    - None
    """
    # plot the nyc streets
    plt.gcf()
    nyc_gdf.plot(linewidth=0.1, edgecolor='black', figsize=(12, 12), alpha=0.5, label="NYC Streets")

    # plot the cluster locations and the pickup locations color-coded
    # to their associated cluster
    plt.scatter(df['pickup_longitude'], df['pickup_latitude'], c=df['pickup_200_cluster'], cmap='magma', alpha=1.0, s=0.1, label="Pickup Locations")
    plt.scatter(pickup_200_centers[:, 0], pickup_200_centers[:, 1], c='red', alpha=1, s=10, label="Cluster Centers")
    leg = plt.legend(loc='upper left')
    for lh in leg.legend_handles: 
        lh.set_alpha(1)
    plt.title("200-KMeans Clustering for Pickup Locations")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # save the plot
    plt.savefig("images/kmeans_200_pickup_save.png")
    plt.show() if SHOW_PLOTS else plt.clf()


########################################
# PLOT DROPOFF LOCATIONS WITH CLUSTERS #
########################################
def plot_cluster_dropoff(df, dropoff_200_centers):
    """
    Plots KMeans clustering for dropoff locations.

    Parameters:
    - df (DataFrame): The DataFrame containing dropoff locations and their associated clusters.
    - dropoff_200_centers (array): The cluster centers for dropoff locations.

    Returns:
    - None
    """
    # plot the nyc streets
    plt.gcf()
    nyc_gdf.plot(linewidth=0.1, edgecolor='black', figsize=(12, 12), alpha=0.5, label="NYC Streets")

    # plot the cluster locations and the pickup locations color-coded
    # to their associated cluster
    plt.scatter(df['dropoff_longitude'], df['dropoff_latitude'], c=df['dropoff_200_cluster'], cmap='viridis', alpha=1.0, s=0.1, label="Dropoff Locations")
    plt.scatter(dropoff_200_centers[:, 0], dropoff_200_centers[:, 1], c='blue', alpha=1, s=10, label="Cluster Centers")
    leg = plt.legend(loc='upper left')
    for lh in leg.legend_handles: 
        lh.set_alpha(1)
    plt.title("200-KMeans Clustering for Dropoff Locations")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # save the plot
    plt.savefig("images/kmeans_200_dropoff_save.png")
    plt.show() if SHOW_PLOTS else plt.clf()


## Visualizations

In [ ]:
def plot_actual_vs_predicted_distributions(show=False):
    """
    Plots histograms of actual and predicted trip durations on the test set and compares their distributions.
    
    Parameters:
    - show (bool): If True, displays the plot; if False, saves the plot as 'images/actual_vs_predicted.png'.
    
    Returns:
    - None
    """
    
    # load in the actual and predicted trip durations
    X, y = get_X_y()
    feature_X = generate_features(X, y)
    feature_X = feature_X.drop(columns=['pickup_datetime'])
    X_train, X_test, y_train, y_test = train_test_split(feature_X, y, test_size=0.2, random_state=42)
    
    # plot a histogram of the actual trip duration distribution in the test set
    # and compute the mean
    counts, bins, patches = plt.hist(y_test, label='Actual Trip Durations')
    actual_mean = np.mean(y_test)

    # plot a histogram of the predicted trip duration distribution on the test set
    counts_pred, bins_pred, pathces_pred = plt.hist(y_pred, label='Predicted Trip Durations', color='orange')

    # plot a verticle line at the actual mean of the distribution
    top = max(np.max(counts), np.max(counts_pred))
    plt.vlines([actual_mean], 0, top, color='black', linestyles='dashed', label='Actual Mean')

    # set other plot parameters and show the plot
    plt.legend()
    plt.title("Actual and Estimated Distribution of Trip Durations")
    plt.xlabel("Trip Duration (seconds)")
    plt.ylabel("Frequency")
    plt.savefig('images/actual_vs_predicted.png')
    
    if show:
        plt.show()
    else:
        plt.clf()


def plot_heat_map(show=False):
    """
    Generate and save a heatmap of pickup frequency by day and hour.

    Parameters:
    - show (bool): If True, display the plot; if False (default), save the plot without displaying.

    Returns:
    - None
    """
    # Load in the actual and predicted trip durations
    X, y = get_X_y()
    feature_X = generate_features(X, y)
    
    # Create new column for day of the week
    feature_X['day_of_week'] = feature_X['pickup_datetime'].dt.day_name()

    # Define the order of days
    day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

    # Create a pivot table for day_of_week and pickup_hour
    pivot_table = feature_X.pivot_table(index='day_of_week', columns='pickup_hour', aggfunc='size', fill_value=0)
    
    # Sort index and columns for better visualization
    pivot_table.sort_index(ascending=False, inplace=True)
    pivot_table = pivot_table.reindex(day_order)

    # Create the heatmap using seaborn
    plt.figure(figsize=(10, 4), dpi=200)
    sns.heatmap(pivot_table, cmap='magma', fmt='d', cbar_kws={'label': 'Frequency'})
    plt.title('Pickup Frequency by Day and Hour')
    plt.xlabel('Hour of Day')
    plt.ylabel('Day of Week')
    plt.savefig('images/day_hour.png')
    
    # Show or save the plot based on the 'show' parameter
    if show:
        plt.show()
    else:
        plt.clf()


def plot_residual(y_actual, y_pred, title="Residuals for [Model Name]", save_postfix='model', show=False):
    """
    Plot and optionally save the residuals of a regression model.

    Parameters:
    - y_actual (array-like): The actual values.
    - y_pred (array-like): The predicted values.
    - title (str, optional): Title for the plot. Default is "Residuals for [Model Name]".
    - save_postfix (str, optional): A postfix to add to the saved image filename. Default is 'model'.
    - show (bool, optional): If True, display the plot; if False (default), save the plot without displaying.

    Returns:
    - None
    """
    # Compute the residuals
    residual = y_actual - y_pred
    
    # Plot the residuals with the x-axis as the predictions and the y-axis
    # as the residuals (y_actual - y_pred)
    plt.scatter(y_pred, residual, alpha=0.1, label="Residuals")
    
    # Plot the residual=0 line
    lower = np.min(y_pred)
    upper = np.max(y_pred)
    ls = np.linspace(lower, upper, 100)
    plt.plot(ls, np.zeros(len(ls)), color='black', label='Residual=0 line')
    
    # Plot the mean of the actual data
    plt.vlines([np.mean(y_actual)], ymin=np.min(residual), ymax=np.max(residual), linestyles='dashed', color='red', label='Mean of y_actual')
    
    # Setup other plot parameters and show or save the plot
    plt.xlabel("Prediction $\widehat{y}$")
    plt.ylabel("Residual ( $y$ - $\widehat{y}$ )")
    plt.title(title)
    plt.legend()
    plt.savefig(f'images/residual_plot_{save_postfix}.png')
    
    # Show or save the plot based on the 'show' parameter
    if show:
        plt.show()
    else:
        plt.clf()

### Clustering Plot

In [ ]:
def duration_per_distance():
    """
    Analyzes the relationship between trip duration and distance between pickup and dropoff locations
    for NYC taxi data using clustering and visualization.

    Parameters:
    - None

    Returns:
    - None
    """
    # Retrieve data and preprocess
    X, y = get_X_y()
    X = generate_features(X, y)
    full_df = pd.concat([X, y], axis=1)
    nyc_gdf = get_nyc_gdf()
    
    # create a copy of the dataframe
    df = deepcopy(full_df)
    
    # Load KMeans models for pickup and dropoff locations
    with open("models/kmeans_200_pickup.pkl", "rb") as file:
        kmeans_200_pickup = pickle.load(file)
    with open("models/kmeans_200_dropoff.pkl", "rb") as file:
        kmeans_200_dropoff = pickle.load(file)
        
    # Predict the clusters for pickup and dropoff locations
    df['pickup_200_cluster'] = kmeans_200_pickup.predict(df[['pickup_longitude', 'pickup_latitude']].values)
    df['dropoff_200_cluster'] = kmeans_200_dropoff.predict(df[['dropoff_longitude', 'dropoff_latitude']].values)

    # Retrieve cluster centers
    pickup_200_centers = kmeans_200_pickup.cluster_centers_
    dropoff_200_centers = kmeans_200_dropoff.cluster_centers_
    
    # Create DataFrames for cluster coordinates
    pickup_cluster_df = pd.DataFrame({
        'pickup_200_cluster': np.arange(0, 200),
        'pickup_longitude': pickup_200_centers[:, 0],
        'pickup_latitude': pickup_200_centers[:, 1]
    })
    dropoff_cluster_df = pd.DataFrame({
        'dropoff_200_cluster': np.arange(0, 200),
        'dropoff_longitude': dropoff_200_centers[:, 0],
        'dropoff_latitude': dropoff_200_centers[:, 1]
    })
    
    # compute the average duration from cluster to cluster
    group_durations = (df
        .groupby(['pickup_200_cluster', 'dropoff_200_cluster'])['trip_duration']
        .mean()
        .reset_index()
        .rename(columns={'trip_duration': 'avg_cluster_duration'}))
    
    # Sort and merge cluster durations and coordinates
    group_durations_sorted = group_durations.sort_values(by='avg_cluster_duration', ascending=True)
    cluster_dist = group_durations_sorted.copy()

    cluster_dist = pd.merge(
        left=cluster_dist, right=pickup_cluster_df, how='left',
        left_on=['pickup_200_cluster'], right_on=['pickup_200_cluster'])
    cluster_dist = pd.merge(
        left=cluster_dist, right=dropoff_cluster_df, how='left',
        left_on=['dropoff_200_cluster'], right_on=['dropoff_200_cluster'])

    # Calculate distances between pickup and dropoff locations
    pickup_longitudes = cluster_dist['pickup_longitude'].values
    pickup_latitudes = cluster_dist['pickup_latitude'].values
    dropoff_longitudes = cluster_dist['dropoff_longitude'].values
    dropoff_latitudes = cluster_dist['dropoff_latitude'].values

    cluster_dist['distance'] = np.sqrt((pickup_longitudes - dropoff_longitudes)**2 + (pickup_latitudes - dropoff_latitudes)**2)
    cluster_dist['duration_dist_ratio'] = cluster_dist['avg_cluster_duration'] / cluster_dist['distance']
    cluster_dist = cluster_dist.sort_values(by='duration_dist_ratio', ascending=False)

    # Retrieve and plot clusters with the worst traffic
    n = 1000
    worst_traffic = cluster_dist.iloc[:n, :]

    pickup_longitudes = worst_traffic['pickup_longitude'].values
    pickup_latitudes = worst_traffic['pickup_latitude'].values
    dropoff_longitudes = worst_traffic['dropoff_longitude'].values
    dropoff_latitudes = worst_traffic['dropoff_latitude'].values

    # plot the nyc streets
    fig, ax = plt.subplots()
    fig.set_size_inches(12, 12)

    nyc_gdf.plot(ax=ax, linewidth=0.1, edgecolor='black', alpha=0.5, label="NYC Streets")

    # plot the cluster locations and the pickup locations color-coded
    plt.scatter(pickup_longitudes, pickup_latitudes, c='blue', alpha=1.0, s=10, label="Pickup Locations")
    plt.scatter(dropoff_longitudes, dropoff_latitudes, c='red', alpha=1.0, s=10, label="Dropoff Locations")
        
    # plot the lines between pickup and dropoff locations
    for plong, plat, dlong, dlat in zip(pickup_longitudes, pickup_latitudes, dropoff_longitudes, dropoff_latitudes):
        x = np.linspace(plong, dlong, 100)
        y = np.linspace(plat, dlat, 100)
        cols = np.linspace(0, 1, 100)
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
                                
        lc = LineCollection(segments, cmap='coolwarm')
        lc.set_array(cols)
        lc.set_linewidth(2)
        ax.add_collection(lc)
    
    # set other plot parameters and show the plot
    plt.title(f"{n} Clusters with longest average trip duration per distance", fontsize=20)
    plt.legend(fontsize=16)
    plt.xlabel("Longitude", fontsize=20)
    plt.ylabel("Latitude", fontsize=20)
    plt.show()


### KDE for Hour and Weather Impact

In [ ]:
def plot_kde(X, show=False):
    """
    Plot Kernel Density Estimate (KDE) plots for pickup frequency by hour based on weather conditions.

    Parameters:
    - X (DataFrame): Input DataFrame containing features, including 'precipitation (mm)' and 'pickup_hour'.
    - show (bool, optional): If True, the plot is displayed interactively. If False, the plot is saved to an image file.
                             Default is False.

    Returns:
    - None
    """
    # get all rows where precipitation (mm) = 0
    sunny = X[X['precipitation (mm)'] == 0]['pickup_hour']
    precip = X[X['precipitation (mm)'] > 0]['pickup_hour']

    # plot the kde plots
    plt.figure(figsize=(8, 5), dpi=200)
    sbn.kdeplot(precip, shade=True, label='Rainy', color='blue', bw_adjust=2)
    sbn.kdeplot(sunny, shade=True, label='Sunny', color='orange', bw_adjust=2)

    # set other plot parameters
    plt.title('Pickup Frequency by Hour and Weather')
    plt.xlabel('Hour of Day')
    plt.ylabel('Frequency')
    plt.legend(loc='upper left')

    if show:
        plt.show()
    else:
        plt.savefig('images/kde_plot.png', dpi=200)
        plt.clf()

## Data Cleaning

In [ ]:
"""
this py file contains all of the data loading, cleaning, and saving logic.
The methods will automatically pull in a cached version of the dataframe
unless force_clean=True. If force_clean=True, then the dataframe will be
cleaned and saved to the data folder.
"""

import pandas as pd
import os
import numpy as np
import json
import geopandas as gpd
from shapely.wkt import loads

from config import (
    data_path, cols_to_drop, SET_VENDOR_ID_TO_01,
    PICKUP_TIME_TO_NORMALIZED_FLOAT
)
from py_files.helper_funcs import p


def clean_data(df, df_name, verbose=False):
    """
    Loads in the train.csv and test.csv and cleans them according
    to the constants in config.py. Saves the cleaned dataframes as
    train_clean.csv and test_clean.csv

    Parameters:
    - df (pandas dataframe): The dataframe to be cleaned
    - df_name (str): The name of the dataframe, either 'train' or 'test'
    - verbose (bool): If True, prints out the progress of the cleaning

    Returns:
    - df_clean (pandas dataframe): The cleaned dataframe
    """

    # only keep the relevant columns based on the config
    p("dropping columns") if verbose else None
    curr_cols_to_drop = [c for c in df.columns if c in cols_to_drop]
    df_clean = df.drop(columns=curr_cols_to_drop)
    p() if verbose else None

    # setting vendor_id to a 0 or 1 instead of 1 and 2
    if SET_VENDOR_ID_TO_01:
        p("setting vendor_id to 0 or 1") if verbose else None
        df_clean['vendor_id'] = df_clean['vendor_id'] - 1
        p() if verbose else None

    # Drop rows with trip duration < 60 seconds
    p("dropping rows with trip duration < 60 seconds") if verbose else None
    df_clean = df_clean[df_clean['trip_duration'] >= 60]

    # Drop rows with outlier locations
    p("dropping rows with outlier locations") if verbose else None
    json_file_path = './misc/lat_long_bounds.json'
    # Read in coordinates
    with open(json_file_path, 'r') as json_file:
        # Load the JSON data from the file
        coords = json.load(json_file)
    df_clean = df_clean[(df_clean['pickup_latitude'] >= coords['lat']['min']) & (
        df_clean['pickup_latitude'] <= coords['lat']['max'])]
    df_clean = df_clean[(df_clean['pickup_longitude'] >= coords['lon']['min']) & (
        df_clean['pickup_longitude'] <= coords['lon']['max'])]
    df_clean = df_clean[(df_clean['dropoff_latitude'] >= coords['lat']['min']) & (
        df_clean['dropoff_latitude'] <= coords['lat']['max'])]
    df_clean = df_clean[(df_clean['dropoff_longitude'] >= coords['lon']['min']) & (
        df_clean['dropoff_longitude'] <= coords['lon']['max'])]

    # Keep only <99.5% of trip duration
    p("dropping rows with trip duration > 99.5%") if verbose else None
    df_clean = df_clean[df_clean['trip_duration'] <=
                        df_clean['trip_duration'].quantile(0.995)]

    # Split apart pickup_datetime
    df_clean['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df_clean['pickup_month'] = df_clean['pickup_datetime'].dt.month
    df_clean['pickup_day'] = df_clean['pickup_datetime'].dt.day_name()
    df_clean = pd.get_dummies(
        df_clean, columns=['pickup_day'], drop_first=True)
    df_clean['pickup_hour'] = df_clean['pickup_datetime'].dt.hour
    df_clean['pickup_minute'] = df_clean['pickup_datetime'].dt.minute

    # Create a pickup period.
    df_clean['pickup_period'] = pd.cut(df_clean['pickup_hour'], bins=[
                                       -1, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])

    # Get dummies for the pickup period.
    df_clean = pd.get_dummies(
        df_clean, columns=['pickup_period'], drop_first=True)

    # Add cyclic data.
    df_clean['pickup_hour_sin'] = np.sin(
        2 * np.pi * df_clean['pickup_hour'] / 24)
    df_clean['pickup_hour_cos'] = np.cos(
        2 * np.pi * df_clean['pickup_hour'] / 24)

    # convert pickup and dropoff times to floats from 0 to 1
    if PICKUP_TIME_TO_NORMALIZED_FLOAT:
        df_clean['pickup_datetime_norm'] = pd.to_datetime(
            df_clean['pickup_datetime']).view('int64') // 10**9
        df_clean['pickup_datetime_norm'] = (df_clean['pickup_datetime_norm'] - df_clean['pickup_datetime_norm'].min()) / (
            df_clean['pickup_datetime_norm'].max() - df_clean['pickup_datetime_norm'].min())
    
    # Drop the id column
    df_clean = df_clean.drop(columns=['id'])

    # save the cleaned dataframe
    p("saving cleaned dataframe") if verbose else None
    df_clean.to_csv(f"{data_path}/{df_name}_clean.csv", index=False)
    p() if verbose else None

    return df_clean


def get_train_data(force_clean=False):
    """
    Either creates the cleaned train dataframe from the train.csv
    or it loads it from the data folder

    Parameters:
    - force_clean (bool): If True, forces the data to be cleaned

    Returns:
    - train (pandas dataframe): A dataframe of the train data
    """
    if not os.path.exists(f"{data_path}/train_clean.csv") or force_clean:
        train = pd.read_csv(f"{data_path}/train.csv")
        return clean_data(train, 'train')
    else:
        return pd.read_csv(f"{data_path}/train_clean.csv")


def get_X_y(return_np=False, force_clean=False):
    """
    Returns the X and y dataframes from a dataframe

    Parameters:
    - return_np (bool): If True, returns numpy arrays instead of
    - force_clean (bool): If True, forces the data to be cleaned

    Returns:
    - X (pandas dataframe): A dataframe of the input data
    - y (pandas dataframe): A dataframe of the label data
    """
    df = get_train_data(force_clean=force_clean)
    X = df.drop(columns=['trip_duration'])
    y = df['trip_duration']

    if return_np:
        X, y = X.values, y.values

    return X, y


def get_test_data():
    """
    Either creates the cleaned test dataframe from the test.csv
    or it loads it from the data folder

    Parameters:
    - None

    Returns:
    - test (pandas dataframe): A dataframe of the test data
    """
    if not os.path.exists(f"{data_path}/test_clean.csv"):
        test = pd.read_csv(f"{data_path}/test.csv")
        return clean_data(test, 'test')
    else:
        return pd.read_csv(f"{data_path}/test_clean.csv")


def get_clean_weather():
    """
    Loads in the NYC_Weather_2016_2022.csv and cleans it according
    to the constants in config.py. Saves the cleaned dataframe as
    weather_clean.csv

    Parameters:
    - None

    Returns:
    - weather (pandas dataframe): A dataframe of the weather
    """
    if not os.path.exists(f"{data_path}/weather_clean1.csv"):
        weather = pd.read_csv(f"{data_path}/NYC_Weather_2016_2022.csv")
        weather = weather.dropna()
        weather['time'] = pd.to_datetime(weather['time'])
        weather = weather[weather['time'] <= '2016-07-01']
        weather = weather.drop(columns=['rain (mm)',
                                        'cloudcover_low (%)',
                                        'cloudcover_mid (%)',
                                        'cloudcover_high (%)',
                                        'windspeed_10m (km/h)',
                                        'winddirection_10m (°)'])
        weather.to_csv(f"{data_path}/weather_clean1.csv", index=False)
        return weather
    else:
        return pd.read_csv(f"{data_path}/weather_clean1.csv")


def get_google_distance():
    """
    Loads in the train_distance_matrix.csv and cleans it according
    to the constants in config.py. Saves the cleaned dataframe as
    google_distance_clean.csv

    Parameters:
    - None

    Returns:
    - google_distance (pandas dataframe): A dataframe of the google
    """
    if not os.path.exists(f"{data_path}/google_distance_clean.csv"):
        google_distance = pd.read_csv(f"{data_path}/train_distance_matrix.csv")

        columns_to_keep = ['id', 'gc_distance', 'google_distance']
        google_distance = google_distance[columns_to_keep]

        google_distance.to_csv(
            f"{data_path}/google_distance_clean.csv", index=False)
        return google_distance
    else:
        return pd.read_csv(f"{data_path}/google_distance_clean.csv")
    
    
def get_nyc_gdf():
    """
    Loads in the NYC street centerline data and returns it as a
    geopandas dataframe

    Parameters:
    - None

    Returns:
    - gdf (geopandas dataframe): A geopandas dataframe of the NYC
    """
    nyc_df = pd.read_csv(f"{data_path}/Centerline.csv")
    nyc_df = nyc_df.loc[:, ['the_geom']]

    # Convert the "the_geom" column to Shapely geometries
    nyc_df['the_geom_geopandas'] = nyc_df['the_geom'].apply(loads)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(nyc_df, geometry='the_geom_geopandas')
    
    return gdf


## Feature Engineering

In [ ]:
"""
this py file holds all of the logic behind the feature engineering.
The generate_features function takes in an X and a y, and it adds
feature columns based on the config.features_toggle
"""

from config import features_toggle
from py_files.data_manager import get_clean_weather, get_google_distance
import numpy as np
import pandas as pd
import pickle


def distance(df):
    """
    Calculate the Manhattan distance in kilometers between pickup and dropoff locations
    and add it as a new column 'distance_km' to the DataFrame.

    The Manhattan distance, also known as the L1 distance or taxicab distance, between two points
    on the Earth's surface is calculated by finding the absolute differences between their respective
    longitudes and latitudes and summing them up. This function computes the Manhattan distance
    in kilometers between the pickup and dropoff locations in a DataFrame, assuming a constant
    Earth radius of 6371 kilometers.

    Parameters:
    - df (pandas.DataFrame): A DataFrame containing pickup and dropoff coordinates with columns
                           'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', and 'dropoff_latitude'.

    Returns:
    - pandas.DataFrame: A DataFrame with an additional 'distance_km' column representing the Manhattan
                     distance in kilometers between pickup and dropoff locations.
    """
    # Radius of the Earth in kilometers
    earth_radius_km = 6371.0

    # Get the pickup and dropoff coordinates
    lon1 = df['pickup_longitude']
    lat1 = df['pickup_latitude']
    lon2 = df['dropoff_longitude']
    lat2 = df['dropoff_latitude']

    # Convert latitude and longitude from degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])


    # Calculate the differences in latitude and longitude
    delta_lat = abs(lat1 - lat2)
    delta_lon = abs(lon1 - lon2)

    # Calculate the Manhattan distance in kilometers
    df['distance_km'] = earth_radius_km * (delta_lat + delta_lon)

    return df


def add_weather_feature(df):
    """
    Add weather-related features to a DataFrame by merging it with a weather dataset.

    This function merges the input DataFrame with a weather dataset based on the rounded pickup time
    and adds weather-related features to the DataFrame. It rounds the 'pickup_datetime' column to the
    nearest hour to match the weather data's time resolution.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame containing pickup-related data.

    Returns:
    - pandas.DataFrame: A DataFrame with added weather-related features merged from the weather dataset.
    """
    # Get weather data
    weather = get_clean_weather()
    
    # Round the pickup time to the nearest hour (to merge with weather)
    df['rounded_date'] = pd.to_datetime(df['pickup_datetime']).dt.round('H')
    weather['time'] = pd.to_datetime(weather['time'])

    # Merge with weather
    df = df.merge(weather, left_on='rounded_date', right_on='time')

    # Drop unnecessary columns and return the dataframe
    df = df.drop(columns=['rounded_date', 'time'])
    return df


def add_google_distance(df):
    """
    Add Google Maps distance and duration features to a DataFrame by merging it with a Google Maps dataset.
    
    Parameters
    - df (DataFrame): The input DataFrame containing pickup and dropoff coordinates.
    
    Returns
    - df (DataFrame): The DataFrame with added Google Maps distance and duration features.
    """

    # Get the google distance
    google_distance = get_google_distance()

    # Merge with the dataframe (verify 1:1)
    df = df.merge(google_distance, on='id', validate='1:1')

    return df


def add_avg_cluster_duration(df, y):
    """
    Adds a column 'avg_cluster_duration' to the DataFrame representing the average duration 
    from cluster to cluster based on pickup and dropoff locations.

    Parameters:
    - df (DataFrame): The input DataFrame containing features.
    - y (array-like): The array of target values (trip durations).

    Returns:
    - df (DataFrame): The DataFrame with added 'avg_cluster_duration' column.
    """
    df = df.copy()
    df['trip_duration'] = y
    
    # load kmeans_pickup and kmeans_dropoff from the models folder using pickle
    with open("models/kmeans_200_pickup.pkl", "rb") as file:
        kmeans_200_pickup = pickle.load(file)
    with open("models/kmeans_200_dropoff.pkl", "rb") as file:
        kmeans_200_dropoff = pickle.load(file)
        
    # predict the clusters for the pickup and dropoff locations using the kmeans_pickup and kmeans_dropoff
    df['pickup_200_cluster'] = kmeans_200_pickup.predict(df[['pickup_longitude', 'pickup_latitude']].values)
    df['dropoff_200_cluster'] = kmeans_200_dropoff.predict(df[['dropoff_longitude', 'dropoff_latitude']].values)

    # get the centers
    pickup_200_centers = kmeans_200_pickup.cluster_centers_
    dropoff_200_centers = kmeans_200_dropoff.cluster_centers_

    # compute the average duration from cluster to cluster
    group_durations = (df
        .groupby(['pickup_200_cluster', 'dropoff_200_cluster'])['trip_duration']
        .mean()
        .reset_index()
        .rename(columns={'trip_duration': 'avg_cluster_duration'}))

    # merge the average duration from cluster to cluster with the main dataframe
    df = pd.merge(
        left=df, right=group_durations, how='left',
        left_on=['pickup_200_cluster', 'dropoff_200_cluster'], right_on=['pickup_200_cluster', 'dropoff_200_cluster'])

    # fill the missing values with the mean of the average duration from cluster to cluster
    df['avg_cluster_duration'] = df['avg_cluster_duration'].fillna(df['avg_cluster_duration'].mean())
    df.drop(columns=['pickup_200_cluster', 'dropoff_200_cluster', 'trip_duration'], inplace=True)
    
    return df


def generate_features(df, y=None):
    """
    Generates additional features based on the specified feature toggles and appends them to the DataFrame.

    Parameters:
    - df (DataFrame): The input DataFrame containing base features.
    - y (array-like, optional): The array of target values. Required if 'avg_cluster_duration' feature is enabled.

    Returns:
    - feature_df (DataFrame): The DataFrame with added features.
    """
    # append the features to the dataframe
    feature_df = df

    # add the distance feature
    if features_toggle['distance']:
        feature_df = distance(feature_df)
    
    # add the weather feature
    if features_toggle['weather']:
        feature_df = add_weather_feature(feature_df)

    # add the google distance feature
    if features_toggle['google_distance']:
        feature_df = add_google_distance(feature_df)
        
    # add the avg_cluster_duration feature
    if features_toggle['avg_cluster_duration']:
        # check if y is None
        if y is None:
            raise Exception("y must be passed to generate_features if avg_cluster_duration is True")
        feature_df = add_avg_cluster_duration(feature_df, y)
    
    # return the feature dataframe
    return feature_df